# 1 Read data & basic text prerpocessing

In [2]:
import re
import sys
import shutil as sh
import os

import numpy as np
from tensorflow.contrib import learn

import timeit

import numpy as np
from tensorflow.contrib import rnn
import matplotlib.pyplot as plt
import tensorflow as tf

/Users/Pan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#path = "/home/yingjie/Desktop/Machine_Learning/Assignment4/data/"
#path2 = "/home/yingjie/Desktop/Machine_Learning/Assignment4/data_clean/"
path = "/Users/Pan/Google Drive/Data Science/Machine Learning/ML_Assignment4/data/"
path2 = "/Users/Pan/Google Drive/Data Science/Machine Learning/ML_Assignment4/data_clean/"

#path = "C:/Users/yhao/Desktop/MSDS/2018Spring/MachineLearning/Assignments/Assignment4/data/"
#path2 = "C:/Users/yhao/Desktop/MSDS/2018Spring/MachineLearning/Assignments/Assignment4/data_clean/"

In [4]:
def data_preprocess(path,thefile,num_of_reviews=None):    
    # Replace all numbers with "NUM" special character
    num_dec = re.compile("[\d]+\.[\d]+")
    num_comma = re.compile("[\d]+,[\d]+")
    num_reg = re.compile("[\d]+")

    # Detect non-English characters
    non_eng = re.compile("[^a-zA-Z\n ]")
    
    #open file
    file=open(os.path.join(path,thefile), "r", encoding="iso-8859-15")
    text = file.read().lower()
    file.close()
    
    # Replace various forms of numbers with NUM character
    text = num_dec.sub("NUM", text)
    text = num_comma.sub("NUM", text)
    text = num_reg.sub("NUM", text)

    # Replace all non-English characters with a space
    text = non_eng.sub(" ", text)

    # Replace double spaces with a single space
    text = text.replace("  ", " ")
    
    # Split the file into reviews based on \n
    text = text.split("\n")
    
    if num_of_reviews is None:
        num_of_reviews=len(text)
        
    small_train=''
    for i in range(num_of_reviews):
        small_train+=text[i]+" END "
    small_train=[small_train]
    
    return(small_train)

In [7]:
train=data_preprocess(path,"down_sampled_reviews/train_tiny.txt")
valid=data_preprocess(path,"down_sampled_reviews/valid_tiny.txt",num_of_reviews=10)
#all is too much for validation... did not finish after 15min
allvalid=data_preprocess(path,"down_sampled_reviews/valid_tiny.txt",num_of_reviews=100)

# 2b Create vocabulary through a function

In [8]:
def train_textlst_2_vocab(text,seq_size = 2):   
    max_document_length=len(text[0].split(" "))
    
    ## Create the vocabularyprocessor object, setting the max lengh of the documents.
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
        

    ## Transform the documents using the vocabulary.
    train_transformed = np.array(list(vocab_processor.fit_transform(text)))
    
    vocab_dict = vocab_processor.vocabulary_._mapping
    vocab_dict['END'] = -1

    ## Extract word:id mapping from the object.
    vocab_dict = vocab_processor.vocabulary_

    train_transformed=np.array(list(vocab_processor.fit_transform(text)))
    train_transformed=[np.trim_zeros(train_transformed[0], 'b')] #trim zero at the end
    
    trainX, trainY = [], []
    max_document_length=len(train_transformed[0])
    for i in range(max_document_length): 
        #[0,1,2,3,4,5,-1,1,2,3,4,5]
        if -1 not in train_transformed[0][i:i+seq_size+1]:
            trainX.append(np.expand_dims(train_transformed[0][i:i+seq_size], axis=1).tolist())
            trainY.append(train_transformed[0][i+1:i+seq_size+1])
    
    size_vocab = len(vocab_dict)
    
    return trainX,trainY,size_vocab,vocab_dict,max_document_length

In [9]:
seq_size=2
trainX,trainY,size_vocab,train_vocab,max_document_length=train_textlst_2_vocab(train,seq_size = seq_size)
print(size_vocab)
print("X:\n",'\n'.join([str(item) for item in trainX[0:5]]))
print("Y:\n",'\n'.join([str(item) for item in trainY[0:5]]))

32099
X:
 [[1], [2]]
[[2], [3]]
[[3], [4]]
[[4], [5]]
[[5], [6]]
Y:
 [2 3]
[3 4]
[4 5]
[5 6]
[6 7]


## Use train vacabulary to convert valid dataset

In [10]:
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length,vocabulary=train_vocab)

## Transform the documents using the vocabulary.
train_transformed =np.array(list(vocab_processor.fit_transform(valid)))
train_transformed =[np.trim_zeros(train_transformed[0], 'b')] #trim zero at the end

## Extract word:id mapping from the object.
valid_vocab = vocab_processor.vocabulary_._mapping

validX, validY = [], []
max_document_length=len(train_transformed[0])
for i in range(max_document_length): 
    #[0,1,2,3,4,5,-1,1,2,3,4,5]
    if -1 not in train_transformed[0][i:i+seq_size+1]:
        validX.append(np.expand_dims(train_transformed[0][i:i+seq_size], axis=1).tolist())
        validY.append(train_transformed[0][i+1:i+seq_size+1])

In [15]:
print(valid_vocab.items()==train_vocab._mapping.items() )#match then good
print(size_vocab)
print(len(validX))
print(validX[0:5])
print(validY[0:5])

True
32099
3011
[[[1], [74]], [[74], [11]], [[11], [23213]], [[23213], [51]], [[51], [828]]]
[array([74, 11]), array([   11, 23213]), array([23213,    51]), array([ 51, 828]), array([828, 174])]


## all_valid

In [12]:
## Transform the documents using the vocabulary.
train_transformed =np.array(list(vocab_processor.fit_transform(allvalid)))
train_transformed =[np.trim_zeros(train_transformed[0], 'b')] #trim zero at the end

## Extract word:id mapping from the object.
allvalid_vocab = vocab_processor.vocabulary_._mapping

allvalidX, allvalidY = [], []
max_document_length=len(train_transformed[0])
for i in range(max_document_length): 
    #[0,1,2,3,4,5,-1,1,2,3,4,5]
    if -1 not in train_transformed[0][i:i+seq_size+1]:
        allvalidX.append(np.expand_dims(train_transformed[0][i:i+seq_size], axis=1).tolist())
        allvalidY.append(train_transformed[0][i+1:i+seq_size+1])

In [16]:
print(allvalid_vocab.items()==train_vocab._mapping.items() )#match then good
print(size_vocab)
print(len(allvalidX))
print(allvalidX[0:5])
print(allvalidY[0:5])

True
32099
17858
[[[1], [74]], [[74], [11]], [[11], [23213]], [[23213], [51]], [[51], [828]]]
[array([74, 11]), array([   11, 23213]), array([23213,    51]), array([ 51, 828]), array([828, 174])]


# TF setup

    # output dense layer:
    # convert W from [# hidden, 1] to [# training examples, # hidden, 1]
    # step 1: add a new dimension at index 0 using tf.expand_dims
    #w_exp= tf.expand_dims(W_out,0)
    # step 2: duplicate W for 'num_examples' times using tf.tile
    #W_repeated = tf.tile(w_exp,[num_examples,1,1])
    
    # Dense Layer calculation: 
    # [# training examples, sequence length, # hidden] *
    # [# training examples, # hidden, 1] = [# training examples, sequence length]
    
    # Actually, y_pred: [# training examples, sequence length, 1]
    # Remove last dimension using tf.squeeze
    #y_pred = tf.add(tf.matmul(logit, W_repeated),b_out)
    #y_pred = tf.squeeze(y_pred)    # [# training examples, sequence length]
    #y_pred = tf.reshape(y_pred)
    
    #num_examples= tf.cast(num_examples, tf.float32)

## Compare performance of RNNs, LSTMs, GRUs

In [40]:
import timeit
# Encapsulate the entire prediction problem as a function
def build_and_predict(trainX,trainY,validX,validY,cell,cellType,input_dim=1,hidden_dim=100,seq_size = 12,max_itr=2,batch_size=32):
    
    graph = tf.Graph()
    with graph.as_default():
        # input place holders
        # input Shape: [# training examples, sequence length, # features]
        x = tf.placeholder(tf.float32,[None,seq_size,1])

        # label Shape: [# training examples, sequence length]
        y = tf.placeholder(tf.int32,[None,seq_size])


        num_examples = tf.shape(x)[0]

        # RNN Network
        cell = cell

        # RNN output Shape: [# training examples, sequence length, # hidden] 
        outputs, _ = tf.nn.dynamic_rnn(cell,x,dtype=tf.float32)


        #outputs = tf.reshape(outputs, [num_examples*seq_size, hidden_dim])


        # weights for output dense layer (i.e., after RNN)
        # W shape: [# hidden, 1]
        W_out = tf.Variable(tf.random_normal([hidden_dim,size_vocab]),name="w_out")

        # b shape: [1]
        b_out = tf.Variable(tf.random_normal([size_vocab]),name="b_out")


        # logit shape [# training examples, vocab_size] 
        outputs_reshaped = tf.reshape(outputs, [num_examples*seq_size, hidden_dim])
 

        network_output = ((tf.matmul(outputs_reshaped,W_out) + b_out))
        #print(network_output)

        y_pred = tf.reshape(network_output,[num_examples, seq_size, size_vocab])

        # Cost & Training Step
        #cost = -tf.reduce_sum(y*tf.log(y_pred)) #cross_entropy
        weights = tf.ones([num_examples,seq_size])
        loss = tf.contrib.seq2seq.sequence_loss(y_pred,y,weights) 
        cost = tf.reduce_mean(loss)
        train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
        saver=tf.train.Saver(max_to_keep=4)
     

    batch_num=len(trainX)//batch_size
    
    # Run Session
    with tf.Session(graph=graph) as sess:
        # 1) initialize variables
        #sess.run(tf.global_variables_initializer())

        # 2) OR restore old sess:
        saver.restore(sess, tf.train.latest_checkpoint("model/%s/"%(cellType)))
        print("successfully restored %s checkpoint"%(cellType))
        

        # Run for 1000 iterations (1000 is arbitrary, need a validation set to tune!)
        print('Training  %s ...' %cellType)
        start=timeit.default_timer()

        for i in range(max_itr): 
            print("iter %3d" %(i))  
            print ("max batch num: "+str(batch_num))
            train_err_tot=0
            freq=0
            for NUM in range(1,batch_num):
                batch_trainX=trainX[(NUM-1)*batch_size:NUM*batch_size]
                batch_trainY=trainY[(NUM-1)*batch_size:NUM*batch_size]

                _, train_err = sess.run([train_op,cost],feed_dict={x:batch_trainX,y:batch_trainY})
                train_err_tot+=train_err
                freq+=1
                
                if NUM ==1:
                    print("    batch NUM %6d :     train err= %8.5f" %(NUM,train_err))
                elif  NUM % 500 == 0: 
                    print("    batch NUM %6d :     train err= %8.5f" %(NUM,train_err))
                elif NUM==batch_num-1:
                    print("    batch NUM %6d :     train err= %8.5f" %(NUM,train_err))

                if  NUM % 2000 == 0:  #for large batch num, check test_error.
                    test_err = sess.run(cost,feed_dict={x:validX,y:validY})
                    print("    batch NUM %6d :     **test err for 10 valid reviews**= %8.5f" %(NUM,test_err))

                    modelPath = saver.save(sess,"model/%s/"%(cellType),global_step=i+1)
                    print("model saved:%s"%modelPath)

            print('train err for all train=  %8.5f' % (train_err_tot/freq))
            test_err = sess.run(cost,feed_dict={x:allvalidX,y:allvalidY})
            print('test err for all valid=  %8.5f' % (test_err))
            print('---------------------------------')

            end=timeit.default_timer()
            print("Training time : %10.5f"%(end-start)) 
    
    return predicted_vals

In [36]:
input_dim=1 # dim > 1 for multivariate time series
hidden_dim=100 # number of hiddent units h
max_itr=3 # number of training iterations

RNNcell = rnn.BasicRNNCell(hidden_dim)
LSTMcell = rnn.BasicLSTMCell(hidden_dim)
GRUcell = rnn.GRUCell(hidden_dim)

In [35]:
# Build models and predict on testing data
predicted_vals_rnn=build_and_predict(trainX,trainY,validX,validY,RNNcell,"RNN",input_dim,hidden_dim,seq_size,max_itr)

INFO:tensorflow:Restoring parameters from model/RNN/-1
successfully restored RNN checkpoint
Training  RNN ...
iter   0
max batch num: 19385
    batch NUM      1 :     train err=  9.52273
    batch NUM    500 :     train err=  8.49033
    batch NUM   1000 :     train err=  8.75326
    batch NUM   1500 :     train err=  8.03617
    batch NUM   2000 :     train err=  8.88300
    batch NUM   2000 :     **test err for 10 valid reviews**= 11.68913
model saved:model/RNN/-1
    batch NUM   2500 :     train err= 10.71013
    batch NUM   3000 :     train err= 10.63102
    batch NUM   3500 :     train err= 10.98642
    batch NUM   4000 :     train err= 10.00023
    batch NUM   4000 :     **test err for 10 valid reviews**= 11.41516
model saved:model/RNN/-1
    batch NUM   4500 :     train err= 10.33693
    batch NUM   5000 :     train err= 10.97310
    batch NUM   5500 :     train err= 10.12336
    batch NUM   6000 :     train err=  9.33103
    batch NUM   6000 :     **test err for 10 valid review

NameError: name 'predicted_vals' is not defined

In [38]:
predicted_vals_lstm=build_and_predict(trainX,trainY,validX,validY,LSTMcell,"LSTM",input_dim,hidden_dim,seq_size,max_itr)

Training  LSTM ...
iter   0
max batch num: 19385
    batch NUM      1 :     train err= 13.34032
    batch NUM    500 :     train err=  7.01942
    batch NUM   1000 :     train err=  6.57599
    batch NUM   1500 :     train err=  7.05042
    batch NUM   2000 :     train err=  7.27831
    batch NUM   2000 :     **test err for 10 valid reviews**=  7.66077
model saved:model/LSTM/-1
    batch NUM   2500 :     train err=  7.58156
    batch NUM   3000 :     train err=  7.33651
    batch NUM   3500 :     train err=  7.37868
    batch NUM   4000 :     train err=  7.59008
    batch NUM   4000 :     **test err for 10 valid reviews**=  7.68141
model saved:model/LSTM/-1
    batch NUM   4500 :     train err=  7.30622
    batch NUM   5000 :     train err=  8.42359
    batch NUM   5500 :     train err=  7.29197
    batch NUM   6000 :     train err=  7.30200
    batch NUM   6000 :     **test err for 10 valid reviews**=  7.74773
model saved:model/LSTM/-1
    batch NUM   6500 :     train err=  7.12168
  

NameError: name 'predicted_vals' is not defined

In [39]:
predicted_vals_gru=build_and_predict(trainX,trainY,validX,validY,GRUcell,"GRU",input_dim,hidden_dim,seq_size,max_itr=10)

Training  GRU ...
iter   0
max batch num: 19385
    batch NUM      1 :     train err= 16.18868
    batch NUM    500 :     train err=  6.78237
    batch NUM   1000 :     train err=  6.82297
    batch NUM   1500 :     train err=  7.33361
    batch NUM   2000 :     train err=  7.46101
    batch NUM   2000 :     **test err for 10 valid reviews**=  8.18927
model saved:model/GRU/-1
    batch NUM   2500 :     train err=  8.61533
    batch NUM   3000 :     train err=  9.07620
    batch NUM   3500 :     train err=  8.36464
    batch NUM   4000 :     train err=  8.26129
    batch NUM   4000 :     **test err for 10 valid reviews**=  8.65854
model saved:model/GRU/-1
    batch NUM   4500 :     train err=  8.07491
    batch NUM   5000 :     train err= 10.39746
    batch NUM   5500 :     train err=  7.81593
    batch NUM   6000 :     train err=  8.11095
    batch NUM   6000 :     **test err for 10 valid reviews**=  8.32552
model saved:model/GRU/-1
    batch NUM   6500 :     train err=  7.25184
    ba

NameError: name 'predicted_vals' is not defined